# Initial Setup

In [ ]:
import numpy as np 
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import style
style.use('ggplot')

import seaborn as sns
sns.set()

from IPython.display import HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

from os import listdir
print(listdir("../input"))

import timeit
from tqdm import tqdm

# Training Set

In [ ]:
train_nrows = !wc -l ../input/train.csv
train_nrows_val = int(train_nrows[0].split()[0])
print('{:,} rows'.format(train_nrows_val))

In [ ]:
!head ../input/train.csv

In [ ]:
column_names = !head -n1 ../input/train.csv
print(column_names[0].split(','))

In [ ]:
df_train = pd.read_csv('../input/train.csv', skiprows = 0, nrows=train_nrows_val//100,
                       dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64}) #use chunksize to iterate

In [ ]:
def display_df_with_preset_precision(df, precision):
    curr_precision = pd.get_option("display.precision")
    pd.set_option("display.precision", precision)
    display(df)
    pd.set_option("display.precision", curr_precision)
display_df_with_preset_precision(df_train.head(), 16)

In [ ]:
#keep minimal mem footprint 
try:
    del(df_train)    
except NameError:
    pass

In [ ]:
start_time = timeit.default_timer()
try:
    del(df_train_iter)    
except NameError:
    pass

df_train_iter = pd.read_csv('../input/train.csv', chunksize=train_nrows_val//100,
                       dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64},iterator=True) #use chunksize to iterate
df_after_jump_agg = pd.DataFrame()
for df in df_train_iter:
    df['diff_in_time_to_failure']=df['time_to_failure'].diff()
    df_jumps = df.loc[(df['diff_in_time_to_failure'] > 0)]
    #display(df_jumps)
    df_after_jump_agg=df_after_jump_agg.append(df_jumps)
print('elapsed time: {:.2f} sec'.format(timeit.default_timer()-start_time))    

In [ ]:
print(df_after_jump_agg.shape)
df_after_jump_agg

In [ ]:
start_time = timeit.default_timer()
try:
    del(df_train_iter)    
except NameError:
    pass

df_train_iter = pd.read_csv('../input/train.csv', chunksize=train_nrows_val//100,
                       dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64},iterator=True) #use chunksize to iterate
df_before_jump_agg = pd.DataFrame()
for df in df_train_iter:
    if len(df.index.intersection(df_after_jump_agg.index-1)) > 0:
        try:
            #display(df.loc[df.index.intersection(df_agg.index-1),:])
            df_before_jump_agg=df_before_jump_agg.append(df.loc[df.index.intersection(df_after_jump_agg.index-1),:])
        except KeyError:
            print('KeyError')
            pass
print('elapsed time: {:.2f} sec'.format(timeit.default_timer()-start_time))

In [ ]:
print(df_before_jump_agg.shape)
df_before_jump_agg

In [ ]:
index_ranges = [(ent[0],ent[1]) for ent in zip([0]+list(df_after_jump_agg.index)[:-1],list(df_before_jump_agg.index))]
index_ranges

In [ ]:
train_set_lengths =np.array([ent[1]-ent[0] for ent in zip([0]+list(df_before_jump_agg.index)[:-1],list(df_before_jump_agg.index))])
train_set_lengths

In [ ]:
train_set_lengths/train_set_lengths.mean()

In [ ]:
range_index = 3 #

In [ ]:
start_time = timeit.default_timer()
try:
    del(df_sample)    
except NameError:
    pass
df_sample = pd.read_csv('../input/train.csv', skiprows = index_ranges[range_index][0], nrows= index_ranges[range_index][1]-index_ranges[range_index][0],
                       dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
df_sample.columns=['acoustic_data','time_to_failure']
df_sample['acoustic_data'].plot();
plt.show()
df_sample['time_to_failure'].plot();
plt.show()
print('elapsed time: {:.2f} sec'.format(timeit.default_timer()-start_time))

We have 16 train sequences. We may use multiple ways to separate them into train and validation groups. This way we will be able to efficiently utilize the training data we have

In [ ]:
max_time_to_failure_points = pd.read_csv('../input/train.csv', skiprows = 0, nrows= 1, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})['time_to_failure'].append(df_after_jump_agg['time_to_failure'])
max_time_to_failure_points

In [ ]:
max_time_to_failure_points.values[:-1]

In [ ]:
decline_angle_tangents = np.array([ent[0]/ent[1] for ent in zip(max_time_to_failure_points.values[:-1], max_time_to_failure_points.index[1:])])
print(decline_angle_tangents.mean())
print(decline_angle_tangents.std())

In [ ]:
#keep minimal mem footprint 
try:
    del(df_sample)    
except NameError:
    pass

# Test Set

In [ ]:
test_seg_files = listdir("../input/test")
test_seg_files[:5]

In [ ]:
#!ls -l ../input/test | wc -l
len(test_seg_files)

In [ ]:
os.path.join("../input/test",test_seg_files[0])

In [ ]:
!wc -l {os.path.join("../input/test",test_seg_files[0])}

In [ ]:
!head {os.path.join("../input/test",test_seg_files[0])}

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
def plot_test_seg_by_index(idx):
    df_test_seg = pd.read_csv(os.path.join("../input/test",test_seg_files[idx]), dtype={'acoustic_data': np.int16})
    df_test_seg['acoustic_data'].plot();

In [ ]:
interact(plot_test_seg_by_index, idx=widgets.IntSlider(min=0,max=len(test_seg_files)-1,step=1,value=0));

In [ ]:
#all seg files have the same length: 150,000 samples
seg_files_lengths = !for filename in ../input/test/*; do wc -l $filename; done
{ent.split(' ')[0] for ent in seg_files_lengths}

# Models

Try the following:
1. remove constant bias in train and in test
2. simple dot product
3. fourier (fft)
4. short-time fourier 
5. wavelets
6. voice spectrogrum
7. try to use notebooks used for the detection of gravitation waves
8. xgboost
9. set evaluation criterion


### Correlations

In [ ]:
range_index=0 #first training sequence is the shortest one
df_sample = pd.read_csv('../input/train.csv', skiprows = index_ranges[range_index][0], nrows= index_ranges[range_index][1]-index_ranges[range_index][0],
                       dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
df_sample.columns=['acoustic_data','time_to_failure']
df_sample['acoustic_data'].plot();
plt.show()
print ('time_to_failure decline rate = {:.16f}'.format(df_sample['time_to_failure'][0]/df_sample['time_to_failure'].shape[0]))
df_sample['time_to_failure'].plot();
plt.show()

In [ ]:
df_sample['time_to_failure'][0]/df_sample['time_to_failure'].shape[0]

In [ ]:
df_sample['time_to_failure'].shape[0]

we need to find points of maximum cross-correlation between the segment and the training sequence<br>
To save time, the correlation should be calculated efficiently - this means do not implement it by yourself in python. We need to find best implementation of the correlation function. 

In [ ]:
df_sample['acoustic_data'].head()

In [ ]:
df_sample['acoustic_data'].mean()

In [ ]:
train_values = (df_sample['acoustic_data']-df_sample['acoustic_data'].mean()).values
train_values

In [ ]:
df_test_seg = pd.read_csv(os.path.join("../input/test",test_seg_files[0]), dtype={'acoustic_data': np.int16})
df_test_seg.head()

In [ ]:
df_test_seg['acoustic_data'].mean()

In [ ]:
test_values = (df_test_seg['acoustic_data']-df_test_seg['acoustic_data'].mean()).values

In [ ]:
from scipy import signal

In [ ]:
train_values.shape

In [ ]:
test_values.shape

In [ ]:
signal_corr = signal.correlate(np.square(train_values), np.square(test_values),mode='valid', #full, valid, same
                               method='fft')

In [ ]:
signal_corr.shape

In [ ]:
pd.DataFrame(signal_corr).plot();

In [ ]:
def correlation_with_test_seg_idx(idx):
    df_test_seg = pd.read_csv(os.path.join("../input/test",test_seg_files[idx]), dtype={'acoustic_data': np.int16})
    test_values = (df_test_seg['acoustic_data']-df_test_seg['acoustic_data'].mean()).values
    signal_corr = signal.correlate(np.square(train_values), np.square(test_values),mode='same', method='fft')
    pd.DataFrame(signal_corr).plot();

In [ ]:
interact(correlation_with_test_seg_idx, idx=widgets.IntSlider(min=0,max=len(test_seg_files)-1,step=1,value=0));

In [ ]:
#start_time = timeit.default_timer()
#np_corr = np.correlate(df_sample['acoustic_data'].values, df_test_seg['acoustic_data'].values)
#print('elapsed time: {:.2f} sec'.format(timeit.default_timer()-start_time))    
#this takes too long

### Spectrogram

In [ ]:
from scipy.signal import spectrogram

In [ ]:
M = 1024
N = 1024
freqs, times, Sx = signal.spectrogram(df_test_seg['acoustic_data'].values, fs=1, window='hanning',
                                      nperseg=N, noverlap=M - 100,
                                      detrend=False, scaling='spectrum')

f, ax = plt.subplots(figsize=(4.8, 2.4))
ax.pcolormesh(times, freqs / 1000, 10 * np.log10(Sx), cmap='viridis')
ax.set_ylabel('Frequency [kHz]')
ax.set_xlabel('Time [s]');

In [ ]:
f, t, Sxx = spectrogram(df_test_seg['acoustic_data'].values)
plt.pcolormesh(t, f, 10 * np.log10(Sxx))
plt.show()
plt.plot(Sxx)
#plt.ylabel('Frequency [Hz]')
#plt.xlabel('Time [sec]')
plt.show()

In [ ]:
from skimage import util

M = 1024

slices = util.view_as_windows(df_test_seg['acoustic_data'].values, window_shape=(M,), step=100)
print(f'data shape: {df_test_seg["acoustic_data"].values.shape}, Sliced data shape: {slices.shape}')

In [ ]:
win = np.hanning(M + 1)[:-1]
slices = slices * win

In [ ]:
slices = slices.T
print('Shape of `slices`:', slices.shape)

In [ ]:
spectrum = np.fft.fft(slices, axis=0)[:M // 2 + 1:-1]
spectrum = np.abs(spectrum)

In [ ]:
L=df_test_seg['acoustic_data'].values.shape[0]

In [ ]:
rate=10

In [ ]:
f, ax = plt.subplots(figsize=(4.8, 2.4))

S = np.abs(spectrum)
S = 20 * np.log10(S / np.max(S))

ax.imshow(S, origin='lower', cmap='viridis',
          extent=(0, L, 0, rate / 2 / 1000))
ax.axis('tight')
ax.set_ylabel('Frequency [kHz]')
ax.set_xlabel('Time [s]');

https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.fft.html <br>
https://docs.scipy.org/doc/scipy/reference/tutorial/fftpack.html

In [ ]:
#df_test_seg['acoustic_data'].values
from scipy.fftpack import fft
# Number of sample points
#N = 600
# sample spacing
#T = 1.0 / 800.0
#x = np.linspace(0.0, N*T, N)
#y = np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)

#yf = fft(y)
yf = fft(df_test_seg['acoustic_data'].values)
#df_test_seg['acoustic_data'].values

#xf = np.linspace(0.0, 1.0/(2.0*T), N//2)

#plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
plt.plot(np.abs(yf))
plt.grid()
plt.show()

In [ ]:
from scipy import fftpack

In [ ]:
from scipy.signal import cwt

# Evaluation